# Part C1: Softmax Derivative (Case i=j)

## Introduction

**StatQuest Style**: "Time to calculate derivatives... step-by-step!"

### What We'll Cover

1. Why we need the derivative
2. The quotient rule
3. Case i=j derivation
4. Beautiful result: S_i(1 - S_i)

**Let's derive!**

In [1]:
import numpy as np
import matplotlib.pyplot as plt

print('✓ Ready for calculus!')

✓ Ready for calculus!


# The Derivative Formula

```
┌─────────────────────────────────────────────────────────────┐
│         SOFTMAX DERIVATIVE (CASE i=j)                       │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  Softmax: S_i = e^(x_i) / Σ e^(x_j)                        │
│                                                             │
│  Want: ∂S_i/∂x_i (derivative w.r.t. its own input)         │
│                                                             │
│  Use Quotient Rule: (f/g)' = (f'g - fg')/g²                │
│                                                             │
│  f = e^(x_i)     →  f' = e^(x_i)                           │
│  g = Σ e^(x_j)   →  g' = e^(x_i)                           │
│                                                             │
│  ∂S_i/∂x_i = [e^(x_i)·Σe^(x_j) - e^(x_i)·e^(x_i)] / (Σe^(x_j))²  │
│            = e^(x_i)/Σe^(x_j) · [Σe^(x_j) - e^(x_i)] / Σe^(x_j)  │
│            = S_i · (1 - S_i)                                │
│                                                             │
│  Result: ∂S_i/∂x_i = S_i(1 - S_i)                          │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

In [2]:
def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / np.sum(exp_x)

def softmax_derivative_i_equals_j(S_i):
    """Derivative when i=j: S_i(1 - S_i)"""
    return S_i * (1 - S_i)

# Example
logits = np.array([2.0, 1.0, 0.1])
probs = softmax(logits)

print('SOFTMAX DERIVATIVE (i=j)')
print('='*60)
print(f'Probabilities: {probs}')
print('\nDerivatives (∂S_i/∂x_i):')
for i, p in enumerate(probs):
    deriv = softmax_derivative_i_equals_j(p)
    print(f'  Class {i}: {p:.4f} × (1 - {p:.4f}) = {deriv:.4f}')
print('='*60)

SOFTMAX DERIVATIVE (i=j)
Probabilities: [0.65900114 0.24243297 0.09856589]

Derivatives (∂S_i/∂x_i):
  Class 0: 0.6590 × (1 - 0.6590) = 0.2247
  Class 1: 0.2424 × (1 - 0.2424) = 0.1837
  Class 2: 0.0986 × (1 - 0.0986) = 0.0889


# Summary

**Key Result**: ∂S_i/∂x_i = S_i(1 - S_i)

This is the derivative when we differentiate a softmax output with respect to its own input.

**Next**: Part C2 covers case i≠j: ∂S_i/∂x_j = -S_i × S_j

**Double Bam!** 💥💥

# Understanding the Softmax Jacobian Matrix

## What is a Jacobian?

**Simple Definition:** A Jacobian is a matrix of **all partial derivatives** when you have:
- **Multiple inputs** (a vector)
- **Multiple outputs** (another vector)

It shows how **each output** changes with respect to **each input**.

---

## Softmax: A Quick Recap

### Softmax Function:
```
Given logits: z = [z₁, z₂, z₃, ..., zₙ]

Softmax output: p = [p₁, p₂, p₃, ..., pₙ]

Where: pᵢ = e^(zᵢ) / Σⱼ e^(zⱼ)
```

### Key Property:
**All probabilities sum to 1:** Σᵢ pᵢ = 1

This creates **interdependence** - changing one logit affects ALL probabilities!

---

## The Jacobian Matrix: What Does It Represent?

### Structure:
```
         ∂p₁/∂z₁   ∂p₁/∂z₂   ∂p₁/∂z₃   ...   ∂p₁/∂zₙ
         ∂p₂/∂z₁   ∂p₂/∂z₂   ∂p₂/∂z₃   ...   ∂p₂/∂zₙ
J =      ∂p₃/∂z₁   ∂p₃/∂z₂   ∂p₃/∂z₃   ...   ∂p₃/∂zₙ
         ...       ...       ...       ...   ...
         ∂pₙ/∂z₁   ∂pₙ/∂z₂   ∂pₙ/∂z₃   ...   ∂pₙ/∂zₙ
```

### Reading the Matrix:
- **Row i:** How probability pᵢ changes when we change each logit
- **Column j:** How all probabilities change when we change logit zⱼ
- **Element (i,j):** How probability pᵢ changes when we change logit zⱼ

---

## Two Cases: Diagonal vs Off-Diagonal

### Case 1: Diagonal Elements (i = j)
**Question:** How does pᵢ change when we change its own logit zᵢ?

```
∂pᵢ/∂zᵢ = pᵢ(1 - pᵢ)
```

**Intuition:**
- Increasing zᵢ **increases** pᵢ (positive derivative)
- But it's **dampened** by (1 - pᵢ) because probabilities must sum to 1
- If pᵢ is already large (close to 1), increasing zᵢ has less effect

**Example:**
```
If pᵢ = 0.7:
∂pᵢ/∂zᵢ = 0.7 × (1 - 0.7) = 0.7 × 0.3 = 0.21

If pᵢ = 0.1:
∂pᵢ/∂zᵢ = 0.1 × (1 - 0.1) = 0.1 × 0.9 = 0.09
```

---

### Case 2: Off-Diagonal Elements (i ≠ j)
**Question:** How does pᵢ change when we change a *different* logit zⱼ?

```
∂pᵢ/∂zⱼ = -pᵢ × pⱼ
```

**Intuition:**
- Increasing zⱼ **decreases** pᵢ (negative derivative)
- This is the "competition" effect - making one probability larger forces others smaller
- The effect is proportional to both pᵢ and pⱼ

**Example:**
```
If pᵢ = 0.3 and pⱼ = 0.5:
∂pᵢ/∂zⱼ = -0.3 × 0.5 = -0.15

Meaning: Increasing zⱼ by 1 decreases pᵢ by approximately 0.15
```

---

## Visual Example: 3-Class Softmax

### Given:
```
Logits: z = [2.0, 1.0, 0.5]
Probabilities: p = [0.659, 0.242, 0.099]
```

### The Jacobian Matrix:

```
         ∂p₁/∂z₁        ∂p₁/∂z₂        ∂p₁/∂z₃
J =      ∂p₂/∂z₁        ∂p₂/∂z₂        ∂p₂/∂z₃
         ∂p₃/∂z₁        ∂p₃/∂z₂        ∂p₃/∂z₃
```

### Computing Each Element:

**Diagonal (i = j):**
```
J[0,0] = p₁(1-p₁) = 0.659 × (1-0.659) = 0.659 × 0.341 = 0.225
J[1,1] = p₂(1-p₂) = 0.242 × (1-0.242) = 0.242 × 0.758 = 0.183
J[2,2] = p₃(1-p₃) = 0.099 × (1-0.099) = 0.099 × 0.901 = 0.089
```

**Off-Diagonal (i ≠ j):**
```
J[0,1] = -p₁×p₂ = -0.659 × 0.242 = -0.159
J[0,2] = -p₁×p₃ = -0.659 × 0.099 = -0.065
J[1,0] = -p₂×p₁ = -0.242 × 0.659 = -0.159
J[1,2] = -p₂×p₃ = -0.242 × 0.099 = -0.024
J[2,0] = -p₃×p₁ = -0.099 × 0.659 = -0.065
J[2,1] = -p₃×p₂ = -0.099 × 0.242 = -0.024
```

### Final Jacobian:
```
       z₁       z₂       z₃
p₁  [ 0.225  -0.159  -0.065]
p₂  [-0.159   0.183  -0.024]
p₃  [-0.065  -0.024   0.089]
```

---

## Interpreting the Jacobian

### Row 1 (How p₁ changes):
```
[ 0.225  -0.159  -0.065]
```
- Increasing z₁ by 1 → p₁ increases by ~0.225 ✓
- Increasing z₂ by 1 → p₁ decreases by ~0.159 ✗
- Increasing z₃ by 1 → p₁ decreases by ~0.065 ✗

### Column 1 (Effect of changing z₁):
```
[ 0.225]
[-0.159]
[-0.065]
```
- Increasing z₁ by 1:
  - p₁ increases by ~0.225 ✓
  - p₂ decreases by ~0.159 ✗
  - p₃ decreases by ~0.065 ✗

**Notice:** The changes sum to approximately 0 (because probabilities must sum to 1)!

---

## Key Properties of Softmax Jacobian

### Property 1: Diagonal Elements are Positive
```
∂pᵢ/∂zᵢ = pᵢ(1 - pᵢ) > 0  (always positive)
```
**Meaning:** Increasing a logit always increases its own probability.

### Property 2: Off-Diagonal Elements are Negative
```
∂pᵢ/∂zⱼ = -pᵢ × pⱼ < 0  (always negative, when i≠j)
```
**Meaning:** Increasing one logit always decreases other probabilities.

### Property 3: Rows Sum to Zero
```
Σⱼ ∂pᵢ/∂zⱼ = 0
```
**Meaning:** The total change in pᵢ across all logit changes is zero (conservation of probability).

### Property 4: Columns Sum to Zero
```
Σᵢ ∂pᵢ/∂zⱼ = 0
```
**Meaning:** When you change one logit, the total probability change across all outputs is zero.

---

## Why Do We Need the Jacobian?

### In Backpropagation:

When computing gradients through softmax, we need to know:
> "How does the loss change with respect to the logits?"

Using the chain rule:
```
∂Loss/∂zⱼ = Σᵢ (∂Loss/∂pᵢ) × (∂pᵢ/∂zⱼ)
            ↑                  ↑
      From next layer    From Jacobian
```

The Jacobian tells us how to propagate gradients from probability space back to logit space.

---

## Practical Example: Gradient Flow

### Setup:
```
Logits: z = [2.0, 1.0, 0.5]
Probabilities: p = [0.659, 0.242, 0.099]
True label: class 0 (one-hot: [1, 0, 0])

Loss gradient w.r.t. probabilities:
∂Loss/∂p = [p₁-1, p₂-0, p₃-0] = [-0.341, 0.242, 0.099]
```

### Computing ∂Loss/∂z₁:
```
∂Loss/∂z₁ = (∂Loss/∂p₁)×(∂p₁/∂z₁) + (∂Loss/∂p₂)×(∂p₂/∂z₁) + (∂Loss/∂p₃)×(∂p₃/∂z₁)
          = (-0.341)×(0.225) + (0.242)×(-0.159) + (0.099)×(-0.065)
          = -0.077 - 0.038 - 0.006
          = -0.121
```

This tells us: **Decrease z₁ by 0.121** to reduce the loss.

---

## Code Breakdown

```python
def softmax_jacobian(probs):
    """Complete Jacobian matrix"""
    n = len(probs)
    jacobian = np.zeros((n, n))
    
    for i in range(n):           # For each output probability
        for j in range(n):       # For each input logit
            if i == j:
                # Diagonal: how pᵢ changes w.r.t. its own logit zᵢ
                jacobian[i, j] = probs[i] * (1 - probs[i])
            else:
                # Off-diagonal: how pᵢ changes w.r.t. other logit zⱼ
                jacobian[i, j] = -probs[i] * probs[j]
    
    return jacobian
```

---

## Visualization: Heatmap Interpretation

When you visualize the Jacobian as a heatmap:

- **Bright diagonal:** Strong positive derivatives (self-reinforcement)
- **Dark off-diagonal:** Negative derivatives (competition)
- **Symmetry:** J[i,j] and J[j,i] have the same magnitude (but different meaning!)

---

## Summary

### The Softmax Jacobian Matrix:

| Element | Formula | Meaning | Sign |
|---------|---------|---------|------|
| **Diagonal** (i=j) | pᵢ(1-pᵢ) | How pᵢ responds to its own logit zᵢ | Positive ✓ |
| **Off-diagonal** (i≠j) | -pᵢ×pⱼ | How pᵢ responds to other logit zⱼ | Negative ✗ |

### Key Insights:

1. **Softmax creates interdependence** - changing one logit affects ALL probabilities
2. **Diagonal elements** show self-reinforcement (positive feedback)
3. **Off-diagonal elements** show competition (negative feedback)
4. **Conservation property** - rows and columns sum to zero
5. **Essential for backpropagation** - converts probability gradients to logit gradients

### The Big Picture:

> The Jacobian captures the **complete sensitivity** of the softmax function.
> It's the mathematical description of how the "probability distribution" responds
> to changes in the "raw scores" (logits).

🎯 **In neural networks, this is how we backpropagate through the final classification layer!**